In [46]:
import os
import glob
import json
import csv
import json
import nltk
import regex as re
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [47]:
reddit = pd.read_csv('subs_csv_data/boston.csv')
print(reddit.shape)
reddit.head(10)

(47879, 11)


,submission_id,num_crossposts,gilded,can_gild,submission_text,contest_mode,brand_safe,score,submission_title,submission_comments,submission_created_date
0,e4b472,0,0,True,"New grad from NY, just moved here for a job of...",False,True,4,Just moved here and very nervous,Sounds like maybe you should get a t pass | Us...,2019-12-01 05:11:20
1,e4facd,0,0,True,[removed],False,True,0,WHAT AAL REFERS TO,American Alcohol League. It’s the drinking org...,2019-12-01 10:03:39
2,e4hi1g,0,0,False,They are asking people to do that because flig...,False,True,0,Check your flights and call ahead before headi...,Thank goodness you are looking out for people....,2019-12-01 13:34:52
3,e4i294,0,0,True,"I’ve never taken the green line when it snows,...",False,True,0,How bad is the green line going to be tomorrow...,[deleted] | It won't be anything out of the or...,2019-12-01 14:26:48
4,e4i2fw,0,0,True,First winter where I’ll keep my car in the cit...,False,True,0,Threshold for Boston snow emergency/parking ban?,"Not anymore. A few years ago, there was a park...",2019-12-01 14:27:16
5,e4inyo,0,0,True,I'm considering switching from my expensive Sp...,False,True,5,Anyone have experience with discount phone pla...,How much do you pay now? How much are you tryi...,2019-12-01 15:20:08
6,e4iwas,0,0,True,Hey everyone! Visiting Boston this week for th...,False,True,0,Visiting Boston for the first time,Your post appears to be one of a number of com...,2019-12-01 15:38:46
7,e4jl6n,0,0,False,[deleted],False,True,0,Got everything I need to ride the storm out,NaN,2019-12-01 16:31:06
8,e4k5op,0,0,True,We moved here a while ago and everyone raves a...,False,True,29,Can someone explain the appeal of steak tips?,"I dunno, I think they suck too and I've been l...",2019-12-01 17:09:56
9,e4kc5s,0,0,True,I know this is hardly news to anyone in Boston...,False,True,168,Huge pothole on Storrow Drive East right after...,It's just pathetic how a first-world country c...,2019-12-01 17:22:15


In [48]:
from datetime import datetime
reddit['submission_comments'] = reddit['submission_comments'].astype(str)
# Function to determine the season based on the month
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

# Function to determine the part of the day
def get_part_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'


def count_comments(text):
    # Split the text by the delimiter and count the parts
    return len(text.split('|')) if text else 0

# Convert 'submission_created_date' to datetime and create the new columns
reddit['submission_created_date'] = pd.to_datetime(reddit['submission_created_date'])
reddit['Season'] = reddit['submission_created_date'].apply(lambda x: get_season(x.month))
reddit['Part_of_Day'] = reddit['submission_created_date'].apply(lambda x: get_part_of_day(x.hour))


# Apply the function to the 'submission_comments' column to create a new 'num_of_comments' column
reddit['num_of_comments'] = reddit['submission_comments'].apply(count_comments)

In [49]:
reddit['submission_comments']= reddit['submission_comments'].fillna(' ')
# Update the pattern to include [deleted by user] and other unwanted text
pattern = r'\[removed\]|\[deleted\]|\[deleted by user\]|I am a bot, and this action was performed automatically. Please \[contact the moderators of this subreddit\]\(\/message\/compose\/\?to=\/r\/boston\) if you have any questions or concerns.'

def filter_comments(comment_series):   

    bot_message = "I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/boston) if you have any questions or concerns."

    def filter_pattern(comment):
        # Check for specific unwanted text patterns
        if comment in ["[removed]", "[deleted]", "[deleted by user]", bot_message]:
            return False
        return not re.search(pattern, comment)

    # Apply the filtering to each comment after handling null values
    return comment_series.apply(lambda x: ' | '.join([c for c in x.split(' | ') if filter_pattern(c)]))

# Assuming reddit DataFrame is already loaded with a 'submission_comments' column
reddit['submission_comments'] = filter_comments(reddit['submission_comments'])



In [50]:
def clean_data(df):
    # Define the pattern for [removed] and [deleted]
    remove_pattern = r'\[removed\]|\[deleted\]|\[deleted by user\]'
    
    # Replace NaN values with space in 'submission_text' and 'submission_title'
    df['submission_text'] = df['submission_text'].fillna(' ')
    df['submission_title'] = df['submission_title'].fillna(' ')
    
    # Replace the pattern with space in 'submission_text' and 'submission_title'
    df['submission_text'] = df['submission_text'].str.replace(remove_pattern, ' ', regex=True)
    df['submission_title'] = df['submission_title'].str.replace(remove_pattern, ' ', regex=True)
    
    return df

# Apply the function to the DataFrame
reddit = clean_data(reddit)


In [51]:
reddit.to_csv('preprocessed_data/preprocessed_boston.csv', index=False)
reddit.shape

(47879, 14)

In [52]:
reddit.head()

,submission_id,num_crossposts,gilded,can_gild,submission_text,contest_mode,brand_safe,score,submission_title,submission_comments,submission_created_date,Season,Part_of_Day,num_of_comments
0,e4b472,0,0,True,"New grad from NY, just moved here for a job of...",False,True,4,Just moved here and very nervous,Sounds like maybe you should get a t pass | Us...,2019-12-01 05:11:20,Winter,Morning,44
1,e4facd,0,0,True,,False,True,0,WHAT AAL REFERS TO,American Alcohol League. It’s the drinking org...,2019-12-01 10:03:39,Winter,Morning,7
2,e4hi1g,0,0,False,They are asking people to do that because flig...,False,True,0,Check your flights and call ahead before headi...,Thank goodness you are looking out for people....,2019-12-01 13:34:52,Winter,Afternoon,21
3,e4i294,0,0,True,"I’ve never taken the green line when it snows,...",False,True,0,How bad is the green line going to be tomorrow...,It won't be anything out of the ordinary. Snow...,2019-12-01 14:26:48,Winter,Afternoon,9
4,e4i2fw,0,0,True,First winter where I’ll keep my car in the cit...,False,True,0,Threshold for Boston snow emergency/parking ban?,"Not anymore. A few years ago, there was a park...",2019-12-01 14:27:16,Winter,Afternoon,5


In [53]:

# Import the EMOTICONS_EMO dictionary
from emo_unicode import EMOTICONS_EMO


# Load the dataset
reddit = pd.read_csv('preprocessed_data/preprocessed_boston.csv')

reddit['submission_comments']= reddit['submission_comments'].fillna(' ')
reddit['submission_text']= reddit['submission_text'].fillna(' ')
reddit['submission_title']= reddit['submission_title'].fillna(' ')

reddit['combined_text'] = (reddit['submission_title'].astype(str) + ' ' +
                           reddit['submission_text'].astype(str) + ' ' +
                           reddit['submission_comments'].astype(str))

# Define convert_emojis function
def convert_emojis(text):
    # Step 1: Replace emojis with text using the EMOTICONS_EMO dictionary
    for emot in EMOTICONS_EMO:
        text = re.sub(re.escape(emot), ' '.join(EMOTICONS_EMO[emot].replace(",", "").replace(":", "").split()), text)
    
    # Step 2: Remove any remaining emojis using regex
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    return text

# Define clean_social_media_text function
def clean_social_media_text(text):
    text = convert_emojis(text)  # Convert emojis to text

    # Replace '|' with a space
    text = text.replace('|', ' ')

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove user mentions (@username)
    text = re.sub(r'\@\w+', '', text)

    # Remove hashtags (#hashtag)
    text = re.sub(r'\#\w+', '', text)

    # Remove HTML entities (like &amp;)
    text = re.sub(r'\&\w+;', '', text)

    # Replace newline and tab characters with a space
    text = re.sub(r'[\n\r\t]+', ' ', text)
    
    text = re.sub(r'\bnan\b', ' ', text, flags=re.IGNORECASE)

    # Replace multiple spaces with a single space
    # This also handles extra white spaces within and between sentences
    text = re.sub(r'\s+', ' ', text).strip()
    

    # Strip leading and trailing spaces
    text = text.strip()
    text = text.lower()

    return text

# Apply convert_emojis and clean_social_media_text to combined_text
reddit['combined_text'] = reddit['combined_text'].apply(clean_social_media_text)


In [54]:
reddit.to_csv('preprocessed_data/preprocessed_boston.csv', index=False)
reddit.shape

(47879, 15)

In [55]:
reddit.head()

,submission_id,num_crossposts,gilded,can_gild,submission_text,contest_mode,brand_safe,score,submission_title,submission_comments,submission_created_date,Season,Part_of_Day,num_of_comments,combined_text
0,e4b472,0,0,True,"New grad from NY, just moved here for a job of...",False,True,4,Just moved here and very nervous,Sounds like maybe you should get a t pass | Us...,2019-12-01 05:11:20,Winter,Morning,44,just moved here and very nervous new grad from...
1,e4facd,0,0,True,,False,True,0,WHAT AAL REFERS TO,American Alcohol League. It’s the drinking org...,2019-12-01 10:03:39,Winter,Morning,7,what aal refers to american alcohol league. it...
2,e4hi1g,0,0,False,They are asking people to do that because flig...,False,True,0,Check your flights and call ahead before headi...,Thank goodness you are looking out for people....,2019-12-01 13:34:52,Winter,Afternoon,21,check your flights and call ahead before headi...
3,e4i294,0,0,True,"I’ve never taken the green line when it snows,...",False,True,0,How bad is the green line going to be tomorrow...,It won't be anything out of the ordinary. Snow...,2019-12-01 14:26:48,Winter,Afternoon,9,how bad is the green line going to be tomorrow...
4,e4i2fw,0,0,True,First winter where I’ll keep my car in the cit...,False,True,0,Threshold for Boston snow emergency/parking ban?,"Not anymore. A few years ago, there was a park...",2019-12-01 14:27:16,Winter,Afternoon,5,threshold for boston snow emergency/parking ba...
